In [2]:
import matplotlib

# import setup_functions as mainmod
# import load_plot_save as load
# import numerical_methods as mantle_timestepping
# import analysis
# import core_function
# import mantle_properties


import pytesimal.setup_functions as mainmod
import pytesimal.load_plot_save as load
import pytesimal.numerical_methods as mantle_timestepping
import pytesimal.analysis as analysis
import pytesimal.core_function as core_function
import pytesimal.mantle_properties as mantle_properties

%matplotlib inline

In [5]:
load.make_default_param_file()

(run_ID, folder, timestep, r_planet, core_size_factor,
reg_fraction, max_time, temp_core_melting, olivine_cp,
olivine_density, cmb_conductivity, core_cp, core_density,
temp_init, temp_surface, core_temp_init, core_latent_heat,
kappa_reg, dr, cond_constant, density_constant,
heat_cap_constant) = load.load_params_from_file()

# uncomment if you want to save the file
folder = 'workflow'

In [6]:
load.check_folder_exists(folder)

In [7]:
(r_core,
radii,
core_radii,
reg_thickness,
where_regolith,
times,
mantle_temperature_array,
core_temperature_array) = mainmod.set_up()

latent = []

In [8]:
core_values = core_function.IsothermalEutecticCore(initial_temperature=core_temp_init, melting_temperature=temp_core_melting,
                                                            outer_r=r_core, inner_r=0, rho=core_density, cp=core_cp,
                                                            core_latent_heat=core_latent_heat)

In [9]:
(mantle_conductivity, mantle_heatcap, mantle_density) = mantle_properties.set_up_mantle_properties()


In [10]:
top_mantle_bc = mantle_timestepping.surface_dirichlet_bc
bottom_mantle_bc = mantle_timestepping.cmb_dirichlet_bc

In [ ]:
(mantle_temperature_array,
core_temperature_array,
latent,
)= mantle_timestepping.discretisation(
    core_values,
    latent,
    temp_init,
    core_temp_init,
    top_mantle_bc,
    bottom_mantle_bc,
    temp_surface,
    mantle_temperature_array,
    dr,
    core_temperature_array,
    timestep,
    r_core,
    radii,
    times,
    where_regolith,
    kappa_reg,
    mantle_conductivity,
    mantle_heatcap,
    mantle_density)


In [ ]:
mantle_temperature_array

In [ ]:
(core_frozen,
 times_frozen,
 time_core_frozen,
 fully_frozen) = analysis.core_freezing(core_temperature_array,
                                        max_time,
                                        times,
                                        latent,
                                        temp_core_melting,
                                        timestep)


In [ ]:
dT_by_dt = analysis.cooling_rate(mantle_temperature_array, timestep)
dT_by_dt_core = analysis.cooling_rate(core_temperature_array, timestep)

In [ ]:
mantle_cooling_rates = dT_by_dt
core_cooling_rates = dT_by_dt_core

In [ ]:
d_im = 147  # cz diameter in nm
d_esq = 158  # cz diameter in nm

In [ ]:
imilac_cooling_rate = analysis.cooling_rate_to_seconds(
    analysis.cooling_rate_cloudyzone_diameter(d_im))
esquel_cooling_rate = analysis.cooling_rate_to_seconds(
    analysis.cooling_rate_cloudyzone_diameter(d_esq))

im_depth, im_x, im_time_core_frozen, im_Time_of_Crossing, im_Critical_Radius = analysis.meteorite_depth_and_timing(
    imilac_cooling_rate,
    mantle_temperature_array,
    dT_by_dt,
    radii,
    r_planet,
    core_size_factor,
    time_core_frozen,
    fully_frozen,
    dr=1000,
)

esq_depth, esq_x, esq_time_core_frozen, esq_Time_of_Crossing, esq_Critical_Radius = analysis.meteorite_depth_and_timing(
    esquel_cooling_rate,
    mantle_temperature_array,
    dT_by_dt,
    radii,
    r_planet,
    core_size_factor,
    time_core_frozen,
    fully_frozen,
    dr=1000,
)

meteorite_results_dict = { 'Esq results':
                               {'depth': esq_depth,
                                'text result': esq_x},
                           'Im results':
                               {'depth' : im_depth,
                                'text result': im_x,
                                'critical radius': im_Critical_Radius}}

In [ ]:
print(f"Imilac depth: {im_depth}; Imilac timing: {im_x}")
print(f"Esquel depth: {esq_depth}; Esquel timing: {esq_x}")

In [ ]:
fig_w = 6
fig_h = 9

load.two_in_one(
    fig_w,
    fig_h,
    mantle_temperature_array,
    core_temperature_array,
    dT_by_dt,
    dT_by_dt_core,)

In [ ]:
result_filename = 'workflow_results1'

In [ ]:
load.save_params_and_results(result_filename, run_ID, folder, timestep, r_planet, core_size_factor,
                        reg_fraction, max_time, temp_core_melting, olivine_cp,
                        olivine_density, cmb_conductivity, core_cp, core_density,
                        temp_init, temp_surface, core_temp_init, core_latent_heat,
                        kappa_reg, dr, cond_constant, density_constant,
                        heat_cap_constant, time_core_frozen, fully_frozen,
                        meteorite_results=meteorite_results_dict)


In [ ]:
load.save_result_arrays(result_filename,
                       folder,
                       mantle_temperature_array,
                       core_temperature_array,
                       mantle_cooling_rates,
                       core_cooling_rates)

